### Multi-group test

In [13]:
ddf_temp = ddf.set_index('tpep_pickup_datetime')
#resample('D')[val_column].mean()

In [ ]:
# TODO modify this to fit in memory - possibly by setting block size in initial loading
ts = 'tpep_pickup_datetime'
def per_group(blk):
    """
    Block is a manageable pandas df for resampling since dask
    will not support it directly
    """
    return blk.groupby(ts)[tip].resample('M', how='sum')

# TODO specify meta param
ddf_temp.groupby(pu_location).apply(per_group).compute()

### Tips with means using dask distributed

In [ ]:
# @dask.delayed
# def test_parallel(ddf):
#     # TODO fix this, worker keeps getting jammed up making things very slow
#     grouping = ddf.groupby([ddf[ts].dt.dayofweek])
#     mean = grouping[tip].mean()
#     std = grouping[tip].std()
#     return mean, std

# %time result1, result2 = test_parallel(ddf).compute()
# %time day_of_week_tips = ddf.groupby([ddf[ts].dt.dayofweek])[tip].mean().compute()

### Grouper seems to have trouble here

In [17]:
time_column =  'tpep_pickup_datetime'
val_column = 'tip_amount'

In [23]:
""" Dynamically set timecourse interval and plot outcome"""
import numpy as np
from pandas import Grouper

In [ ]:
def get_mean_val_by_interval(ddf, time_column, val_column, time_interval = '1min'):
    """
    Get the mean value for a given time interval
    time_column: datetime obj
    val_column: float to average by time_interval
    
    Possible time intervals - https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
    """
    # Have to expand to float64 to prevent overflow and inf outcomes 
    ddf[val_column] = ddf[val_column].astype(np.float64)
#     ddf_by_grouping = ddf.groupby(Grouper(key=time_column, freq=time_interval))
    return ddf.groupby(Grouper(key=time_column, freq=time_interval))[val_column].mean()

get_mean_val_by_interval(ddf_temp, time_column, val_column, time_interval = '1M').compute()